In [2]:
import pandas as pd
import numpy as np
import random
import datetime
import boto3
import botocore
import psycopg2
import configparser

### Nombre de la instancia

In [3]:
rdsIdentifier = 'dpir'

### Cargar archivo de config.

In [4]:
config = configparser.ConfigParser()
config.read('config.cfg')

['config.cfg']

### Crear instancia

In [5]:
aws_data = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                     aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                     region_name='us-east-1')

### Instancias disponibles

In [6]:
rdsInstanceIds = []

response = aws_data.describe_db_instances()
for resp in response['DBInstances']:
    rdsInstanceIds.append(resp['DBInstanceIdentifier'])
    db_instance_status = resp['DBInstanceStatus']

print(f"DBInstanceIds {rdsInstanceIds}")

DBInstanceIds ['dpir']


### Creacion de servicio RDS

In [7]:
try:
    response = aws_data.create_db_instance(
            AllocatedStorage=10,
            DBName=config.get('RDS', 'DB_NAME'),
            DBInstanceIdentifier=rdsIdentifier,
            DBInstanceClass="db.t3.micro",
            Engine="postgres",
            MasterUsername=config.get('RDS', 'DB_USER'),
            MasterUserPassword=config.get('RDS', 'DB_PASSWORD'),
            Port=int(config.get('RDS', 'DB_PORT')),
            VpcSecurityGroupIds=["sg-012fb8317adaa665f"],
            PubliclyAccessible=True
        )
    print (response)
except aws_data.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La Instancia de Base de Datos ya Existe.")

La Instancia de Base de Datos ya Existe.


### Obteniendo URL del host

In [8]:
try:
     instances = aws_data.describe_db_instances(DBInstanceIdentifier=rdsIdentifier)
     RDS_HOST = instances.get('DBInstances')[0].get('Endpoint').get('Address')
     print(RDS_HOST)
except Exception as ex:
     print("La instancia de base de datos no existe o aun no se ha terminado de crear.")
     print(ex)

dpir.cfbnwxvpqafk.us-east-1.rds.amazonaws.com


### Cargar informacion a base de datos

In [ ]:
import DDLPR

try:
    db_conn = psycopg2.connect(
        database=config.get('RDS', 'DB_NAME'), 
        user=config.get('RDS', 'DB_USER'),
        password=config.get('RDS', 'DB_PASSWORD'), 
        host=RDS_HOST,
        port=config.get('RDS', 'DB_PORT')
    )

    cursor = db_conn.cursor()
    cursor.execute(DDLPR.create_pr_dw)
    db_conn.commit()
    print("Base de Datos Creada Exitosamente")
except Exception as ex:
    print("ERROR: Error al crear la base de datos.")
    print(ex)

### Funcion para cargar datos

In [10]:
def insertDataToSQL(data_dict, table_name):
     postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""    
     df_data = pd.DataFrame.from_records(data_dict)
     try:
          response = df_data.to_sql(table_name, postgres_driver, index=False, if_exists='replace')
          print(f'Se han insertado {response} nuevos registros en {table_name}.' )
     except Exception as ex:
          print(ex)

In [11]:
import os
from pathlib import Path
os.getcwd()

'c:\\Users\\pablo\\OneDrive\\Documentos\\MASTER\\Python\\Proyecto2\\Proyecto\\CargaAWS'

In [12]:
import os

# folder path
dir_path = 'Data Tables'

# list to store files
DataTables= []

# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        DataTables.append(path)
print(DataTables)

['dim_adquisicion.csv', 'dim_date.csv', 'dim_divisas.csv', 'dim_entidad.csv', 'dim_evento.csv', 'dim_fuente.csv', 'dim_medida.csv', 'dim_productos.csv', 'dim_proveedor.csv', 'dim_rubro.csv', 'fact_catalogo.csv', 'fact_precio.csv']


In [13]:
for DT in DataTables:
    TN=DT.removesuffix('.csv')
    FD=os.path.join("Data Tables",DT)
    Data=pd.read_csv(FD,sep=';', encoding='Latin1',encoding_errors='backslashreplace')
    Dict=Data.to_dict(orient='records')
    insertDataToSQL(Dict,TN)

Se han insertado 3 nuevos registros en dim_adquisicion.
Se han insertado 287 nuevos registros en dim_date.
Se han insertado 8 nuevos registros en dim_divisas.
Se han insertado 3 nuevos registros en dim_entidad.
Se han insertado 2 nuevos registros en dim_evento.
Se han insertado 6 nuevos registros en dim_fuente.
Se han insertado 5 nuevos registros en dim_medida.
Se han insertado 158 nuevos registros en dim_productos.
Se han insertado 130 nuevos registros en dim_proveedor.
Se han insertado 6 nuevos registros en dim_rubro.
Se han insertado 158 nuevos registros en fact_catalogo.
Se han insertado 939 nuevos registros en fact_precio.
